# New Section



> # Cellule 1 : Téléchargement et Préparation Initiale des Données





In [3]:
import json
import requests
import os

# URL vers le fichier de données brut sur GitHub
data_url = "https://raw.githubusercontent.com/chaimaElouichouany/LLM-Powered-Automated-Incident-Response-and-Playbook-Generation/main/display-all-training-data%20(1).json"

print(f"Téléchargement des données depuis : {data_url}")
# Téléchargement des données
try:
    response = requests.get(data_url)
    response.raise_for_status() # Vérifie si la requête a réussi
    data = response.json()
    print("Données téléchargées avec succès.")

    # Extraction de tous les textes du champ "text"
    all_texts = []
    for item in data:
        if "text" in item:
            all_texts.append(item["text"])

    # Fusion de tous les textes en une seule grande chaîne de caractères, séparés par un saut de ligne
    training_data_string = "\n".join(all_texts)

    # Sauvegarde des données dans input.txt dans le répertoire /content/
    output_initial_path = "/content/input.txt"
    with open(output_initial_path, "w", encoding="utf-8") as f:
        f.write(training_data_string)

    print(f"Données sauvegardées dans {output_initial_path}")
    print(f"Nombre total de caractères dans les données d'entraînement : {len(training_data_string)}")
    print(f"Nombre d'exemples (playbooks) dans les données : {len(all_texts)}")

except requests.exceptions.RequestException as e:
    print(f"Erreur lors du téléchargement des données : {e}")
except json.JSONDecodeError as e:
    print(f"Erreur lors du décodage du JSON : {e}")
except Exception as e:
    print(f"Une erreur inattendue est survenue (Cellule 1) : {e}")

Téléchargement des données depuis : https://raw.githubusercontent.com/chaimaElouichouany/LLM-Powered-Automated-Incident-Response-and-Playbook-Generation/main/display-all-training-data%20(1).json
Données téléchargées avec succès.
Données sauvegardées dans /content/input.txt
Nombre total de caractères dans les données d'entraînement : 362468
Nombre d'exemples (playbooks) dans les données : 49


# Cellule 2 : Clonage de nanoGPT et Installation des Dépendances

In [9]:
import os

print("Clonage du dépôt nanoGPT...")
!git clone https://github.com/karpathy/nanogpt.git

# Déplacement vers le dossier nanogpt
try:
    %cd nanogpt
    print(f"Répertoire actuel : {os.getcwd()}") # os.getcwd() nécessite 'import os'

    print("Installation de tiktoken...")
    !pip install tiktoken
    print("Dépendances installées.")
except Exception as e:
    print(f"Une erreur est survenue lors du clonage ou de l'installation (Cellule 2) : {e}")
    print("Veuillez vérifier si le dépôt a été cloné et si vous êtes dans le bon répertoire.")

Clonage du dépôt nanoGPT...
Cloning into 'nanogpt'...
remote: Enumerating objects: 686, done.
remote: Total 686 (delta 0), reused 0 (delta 0), pack-reused 686 (from 1)
Receiving objects: 100% (686/686), 954.04 KiB | 20.30 MiB/s, done.
Resolving deltas: 100% (387/387), done.
/content/nanogpt/nanogpt
Répertoire actuel : /content/nanogpt/nanogpt
Installation de tiktoken...
Dépendances installées.


# Cellule 3 : Organisation des Données pour nanoGPT

In [10]:
import os
import shutil

expected_base_dir = "/content"
if os.getcwd().endswith("/nanogpt"):
    %cd ..
    print(f"Changé de répertoire vers : {os.getcwd()}")
elif os.getcwd() != expected_base_dir:
    %cd {expected_base_dir}
    print(f"Changé de répertoire vers : {os.getcwd()}")
else:
    print(f"Déjà dans le répertoire : {os.getcwd()}")

nanogpt_base_dir = "/content/nanogpt"
nanogpt_data_dir = os.path.join(nanogpt_base_dir, "data", "incident_playbooks")

print(f"Création du répertoire (s'il n'existe pas) : {nanogpt_data_dir}")
os.makedirs(nanogpt_data_dir, exist_ok=True)

source_input_file = "/content/input.txt"
destination_input_file = os.path.join(nanogpt_data_dir, "input.txt")

print(f"Tentative de déplacement de {source_input_file} vers {destination_input_file}")
if os.path.exists(source_input_file):
    try:
        shutil.move(source_input_file, destination_input_file)
        print("Fichier déplacé avec succès.")
    except Exception as e:
        print(f"Erreur lors du déplacement du fichier avec shutil : {e}")
        print("Tentative avec !mv en fallback...")
        !mv {source_input_file} {destination_input_file}
else:
    print(f"ERREUR : Fichier source {source_input_file} non trouvé. Assurez-vous que la Cellule 1 s'est exécutée correctement.")

# Vérification de l'existence du fichier à la nouvelle destination
if os.path.exists(destination_input_file):
    print(f"Confirmé : {destination_input_file} existe.")
else:
    print(f"ERREUR : {destination_input_file} n'existe pas après l'opération de déplacement.")

/content/nanogpt
Changé de répertoire vers : /content/nanogpt
Création du répertoire (s'il n'existe pas) : /content/nanogpt/data/incident_playbooks
Tentative de déplacement de /content/input.txt vers /content/nanogpt/data/incident_playbooks/input.txt
Fichier déplacé avec succès.
Confirmé : /content/nanogpt/data/incident_playbooks/input.txt existe.


# Cellule 4  : Création du script prepare.py pour votre jeu de données


In [11]:
%%writefile /content/nanogpt/data/incident_playbooks/prepare.py
import os
import pickle
import tiktoken # tiktoken est utilisé pour le BPE de GPT-2
import numpy as np

# Chemin vers le fichier input.txt
input_file_path = os.path.join(os.path.dirname(__file__), 'input.txt')
# Répertoire de sortie
output_dir = os.path.dirname(__file__)

if not os.path.exists(input_file_path):
    print(f"Erreur : {input_file_path} non trouvé.")
    exit()

print(f"Lecture des données depuis {input_file_path}...")
with open(input_file_path, 'r', encoding='utf-8') as f:
    data = f.read()
print(f"Données lues. Longueur totale : {len(data)} caractères.")

# Division des données : 90% pour l'entraînement, 10% pour la validation
n = len(data)
train_data = data[:int(n*0.9)]
val_data = data[int(n*0.9):]

# Encodage avec le tokenizer GPT-2 de tiktoken (BPE)
print("Encodage des données avec tiktoken (gpt2)...")
enc = tiktoken.get_encoding("gpt2")
train_ids = enc.encode_ordinary(train_data) # Utiliser encode_ordinary pour du texte brut
val_ids = enc.encode_ordinary(val_data)
print(f"Entraînement : {len(train_ids):,} tokens")
print(f"Validation : {len(val_ids):,} tokens")

# Exportation vers des fichiers .bin
print("Exportation des IDs de tokens vers des fichiers .bin...")
train_ids_np = np.array(train_ids, dtype=np.uint16)
val_ids_np = np.array(val_ids, dtype=np.uint16)
train_ids_np.tofile(os.path.join(output_dir, 'train.bin'))
val_ids_np.tofile(os.path.join(output_dir, 'val.bin'))
print("train.bin et val.bin créés.")

print("Préparation des données terminée (sans création de meta.pkl).")

Overwriting /content/nanogpt/data/incident_playbooks/prepare.py


# Cellule 5 : Exécution de prepare.py et suppression de l'ancien meta.pkl


In [12]:
import os

prepare_script_dir = "/content/nanogpt/data/incident_playbooks/"
%cd {prepare_script_dir}
print(f"Répertoire actuel : {os.getcwd()}")

print("Exécution de prepare.py (version qui ne devrait plus créer meta.pkl)...")
!python prepare.py
# Suppression explicite de meta.pkl s'il existe d'une exécution précédente
meta_file_path = os.path.join(prepare_script_dir, "meta.pkl")
print(f"Vérification de l'existence du fichier : {meta_file_path}")
if os.path.exists(meta_file_path):
    print(f"Suppression de l'ancien fichier {meta_file_path}...")
    try:
        os.remove(meta_file_path)
        print(f"Fichier {meta_file_path} supprimé avec succès.")
    except Exception as e:
        print(f"Erreur lors de la suppression de {meta_file_path}: {e}")
else:
    print(f"Aucun fichier {meta_file_path} trouvé à supprimer .")

/content/nanogpt/data/incident_playbooks
Répertoire actuel : /content/nanogpt/data/incident_playbooks
Exécution de prepare.py (version qui ne devrait plus créer meta.pkl)...
Lecture des données depuis /content/nanogpt/data/incident_playbooks/input.txt...
Données lues. Longueur totale : 362468 caractères.
Encodage des données avec tiktoken (gpt2)...
Entraînement : 66,404 tokens
Validation : 7,543 tokens
Exportation des IDs de tokens vers des fichiers .bin...
train.bin et val.bin créés.
Préparation des données terminée (sans création de meta.pkl).
Vérification de l'existence du fichier : /content/nanogpt/data/incident_playbooks/meta.pkl
Aucun fichier /content/nanogpt/data/incident_playbooks/meta.pkl trouvé à supprimer .


# Cellule 6 : Création du fichier de configuration d'entraînement


In [13]:
%%writefile /content/nanogpt/config/train_incident_playbooks.py
# Configuration pour l'entraînement de nanoGPT sur les playbooks d'incidents

out_dir = 'out-incident-playbooks-v4'
eval_interval = 500       # Évaluer moins fréquemment pour les longs entraînements
log_interval = 50         # Afficher les logs un peu moins fréquemment
eval_iters = 100
wandb_log = False
wandb_project = 'incident-playbooks'
wandb_run_name = 'nanogpt-playbook-deeper-train'

# --- Données ---
dataset = 'incident_playbooks'
gradient_accumulation_steps = 1
batch_size = 6 # Maintenir à 6 avec block_size=256. Réduire si block_size augmente ou en cas d'erreur mémoire.
block_size = 256 # Maintenir pour l'instant. Peut être augmenté à 512 (avec batch_size réduit)

# --- Paramètres du Modèle ---
n_layer = 6
n_head = 6
n_embd = 384

# --- Paramètres d'Entraînement ---
dropout = 0.1                 # Maintenir à 0.1 pour l'instant
learning_rate = 6e-4          # Taux d'apprentissage légèrement réduit
max_iters = 50000           # Augmentation significative des itérations (جرب 20000-30000)
lr_decay_iters =  50000      # Doit correspondre à max_iters
min_lr = 6e-5                 # 1/10 du taux d'apprentissage initial
compile = False

Overwriting /content/nanogpt/config/train_incident_playbooks.py


# Cellule 7 : Démarrage de l'Entraînement


In [2]:
import os

# Déplacement vers le répertoire principal de nanoGPT
nanogpt_root_dir = "/content/nanogpt/"
%cd {nanogpt_root_dir}
print(f"Répertoire actuel : {os.getcwd()}")

print("Démarrage de l'entraînement ...")
!python train.py config/train_incident_playbooks.py

[Errno 2] No such file or directory: '/content/nanogpt/'
/content
Répertoire actuel : /content
Démarrage de l'entraînement ...
python3: can't open file '/content/train.py': [Errno 2] No such file or directory


# Cellule 8 : Génération de texte à partir du modèle entraîné


In [ ]:
# Cell 8: Sample from the Trained Model
import os

# Make sure this is the directory of the model you want to test
output_directory_name = 'out-incident-playbooks-v4'

checkpoint_path = f"/content/nanogpt/{output_directory_name}/ckpt.pt"

if os.path.exists(checkpoint_path):
    print(f"Testing model from checkpoint: {checkpoint_path}")

    # ---- MODIFY THE PROMPT BELOW FOR EACH TEST ----
    prompt = "<s>[INST] Please generate a detailed playbook for a DDOS  attack [/INST]"

    !python sample.py \
        --out_dir={output_directory_name} \
        --start="{prompt}" \
        --num_samples=1 \
        --max_new_tokens=400 # Increased tokens to see more of the playbook
else:
    print(f"Checkpoint file not found in {checkpoint_path}")

Testing model from checkpoint: /content/nanogpt/out-incident-playbooks-v3/ckpt.pt
Overriding: out_dir = out-incident-playbooks-v3
Overriding: start = <s>[INST] Please generate a detailed playbook for a Ransomware attack [/INST]
Overriding: num_samples = 1
Overriding: max_new_tokens = 400
number of parameters: 29.94M
No meta.pkl found, assuming GPT-2 encodings...
<s>[INST] Please generate a detailed playbook for a Ransomware attack [/INST] Sure. Here is the information requested.
Title: : Incident Response Playbook for Phishing Attack
1. Preparation: : Objective: Establish Contacts, Define Procedures, Gather Information to Save Time During an Incident
1.1 Policies and Procedures
Develop and Document Policies: Create detailed policies and procedures for incident response, specifically for phishing attacks.
Employee Training: Conduct regular training sessions to educate employees on recognizing phishing attempts and the importance of reporting suspicious emails.
Incident Reporting Mechani